# Example of geometry optimization
## Batch of two molecules

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Geometry_Optimization_SD
from seqm.seqm_functions.read_xyz import read_xyz
from seqm.seqm_functions.save_xyz import save_xyz


torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [4]:
%%time

species = torch.as_tensor([[8,6,1,1],
                           [8,6,1,1],
                           [8,1,1,0]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [ 0.00,    0.00,    0.00],
                               [ 0.96,    0.00,    0.00],
                               [-0.45,   -0.57,    0.67],
                               [0.0,0.0,0.0]            # zero-padding for batching
                              ]
                            ], device=device)

const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.008, force_tol=1.0e-2, max_evl=400).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      1.796659e+00 ||-1.362050e+00 -1.362050e+00 ||-1.362050e+00 -1.362050e+00 ||-2.392370e+00 -2.392370e+00 
2      1.400939e+00 ||-1.363489e+00 -1.439058e-03 ||-1.363489e+00 -1.439058e-03 ||-2.449955e+00 -5.758453e-02 
3      1.153429e+00 ||-1.364422e+00 -9.335565e-04 ||-1.364422e+00 -9.335565e-04 ||-2.485697e+00 -3.574174e-02 
4      9.636769e-01 ||-1.364870e+00 -4.476099e-04 ||-1.364870e+00 -4.476099e-04 ||-2.510563e+00 -2.486624e-02 
5      8.074885e-01 ||-1.365146e+00 -2.757984e-04 ||-1.365146e+00 -2.757984e-04 ||-2.528016e+00 -1.745273e-02 
6      6.766222e-01 ||-1.365288e+00 -1.425080e-04 ||-1.365288e+00 -1.425080e-04 ||-2.540260e+00 -1.224393e-02 
7      5.666936e-01 ||-1.365373e+00 -8.517386e-05 ||-1.365373e+00 -8.517386e-05 ||-2.548841e+00 -8.581488e-03 
8      4.744288e-01 ||-1.365419e+00 -4.614317e-05 ||-1.365419e+00 -4.614317e-05 ||-2.554851e+00 -6.010428e-03 
9      3.970775e-01 ||-1.365447e+00 -2.722724e-05 ||-1.365447e+00 -2

### Final forces and optimized geometry

In [7]:
molecule.force

tensor([[[ 9.9667e-03, -2.8699e-14, -4.3301e-15],
         [-9.4592e-03,  1.8697e-13,  2.6211e-15],
         [-2.5373e-04,  1.3308e-03,  4.2835e-15],
         [-2.5373e-04, -1.3308e-03, -2.5745e-15]],

        [[ 9.9667e-03, -2.8699e-14, -4.3301e-15],
         [-9.4592e-03,  1.8697e-13,  2.6211e-15],
         [-2.5373e-04,  1.3308e-03,  4.2835e-15],
         [-2.5373e-04, -1.3308e-03, -2.5745e-15]],

        [[-3.3919e-03,  3.5531e-03, -4.1765e-03],
         [-6.1924e-03, -4.9384e-03,  5.8048e-03],
         [ 9.5843e-03,  1.3853e-03, -1.6283e-03],
         [-0.0000e+00, -0.0000e+00, -0.0000e+00]]], device='cuda:0')

In [8]:
molecule.coordinates

tensor([[[-1.5705e-03, -4.3565e-16,  4.1913e-16],
         [ 1.2257e+00,  1.4420e-15, -1.5915e-16],
         [ 1.8180e+00,  9.3933e-01, -1.7557e-16],
         [ 1.8180e+00, -9.3933e-01, -8.4408e-17]],

        [[-1.5705e-03, -4.3565e-16,  4.1913e-16],
         [ 1.2257e+00,  1.4420e-15, -1.5915e-16],
         [ 1.8180e+00,  9.3933e-01, -1.7557e-16],
         [ 1.8180e+00, -9.3933e-01, -8.4408e-17]],

        [[-3.8492e-02,  2.8403e-02, -3.3386e-02],
         [ 9.1673e-01, -4.1681e-02,  4.8993e-02],
         [-3.6823e-01, -5.5672e-01,  6.5439e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]], device='cuda:0',
       requires_grad=True)

### Save optimized geometries to .xyz files without final forces.

In [9]:
save_xyz(molecule, 'XYZ', Forces=False)

## Reading starting geometries from .xyz
### Only molecules of the same length are currently supported for batched xyz_reader.

In [10]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz', 'coronene.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.012, force_tol=1.0e-2, max_evl=400).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      6.239775e-01 ||4.242794e+00 4.242794e+00 ||4.242794e+00 4.242794e+00 
2      3.792450e-01 ||4.186331e+00 -5.646306e-02 ||4.186331e+00 -5.646306e-02 
3      1.358504e-01 ||4.178392e+00 -7.939291e-03 ||4.178391e+00 -7.939486e-03 
4      1.011021e-01 ||4.176500e+00 -1.891369e-03 ||4.176504e+00 -1.887698e-03 
5      6.815978e-02 ||4.175762e+00 -7.386139e-04 ||4.175764e+00 -7.399696e-04 
6      5.569419e-02 ||4.175298e+00 -4.631112e-04 ||4.175302e+00 -4.619211e-04 
7      4.486743e-02 ||4.174941e+00 -3.577596e-04 ||4.174943e+00 -3.591308e-04 
8      3.946914e-02 ||4.174646e+00 -2.943463e-04 ||4.174647e+00 -2.952524e-04 
9      3.851805e-02 ||4.174399e+00 -2.478058e-04 ||4.174399e+00 -2.481736e-04 
10      3.467146e-02 ||4.174187e+00 -2.118280e-04 ||4.174187e+00 -2.117687e-04 
11      3.326823e-02 ||4.174005e+00 -1.817669e-04 ||4.174005e+00 -1.821376e-04 
12      3.045950e-02 ||4.173848e+00 -1.573237e-04 ||4.173848e+00 -1.574436e-04 
13     

### Alternatively, use zero-padding in .xyz files for molecules of different lengths.

In [11]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz', 'benzene_zero_pad.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.012, force_tol=1.0e-2, max_evl=1000).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      6.239775e-01 ||4.242794e+00 4.242794e+00 ||9.740017e-01 9.740017e-01 
2      3.792450e-01 ||4.186331e+00 -5.646306e-02 ||9.610098e-01 -1.299196e-02 
3      1.609409e-01 ||4.178392e+00 -7.939296e-03 ||9.573780e-01 -3.631755e-03 
4      1.004714e-01 ||4.176500e+00 -1.891303e-03 ||9.560781e-01 -1.299970e-03 
5      6.776770e-02 ||4.175762e+00 -7.384495e-04 ||9.555108e-01 -5.672506e-04 
6      5.517375e-02 ||4.175300e+00 -4.619030e-04 ||9.552377e-01 -2.731449e-04 
7      4.479049e-02 ||4.174942e+00 -3.583238e-04 ||9.551008e-01 -1.369124e-04 
8      3.945767e-02 ||4.174647e+00 -2.946251e-04 ||9.550310e-01 -6.970436e-05 
9      3.848431e-02 ||4.174399e+00 -2.480224e-04 ||9.549954e-01 -3.568108e-05 
10      3.466393e-02 ||4.174187e+00 -2.116342e-04 ||9.549772e-01 -1.821622e-05 
11      3.325615e-02 ||4.174005e+00 -1.820775e-04 ||9.549677e-01 -9.402105e-06 
12      3.045552e-02 ||4.173848e+00 -1.574189e-04 ||9.549629e-01 -4.861493e-06 
13     